<H1>PROJECT BATON V2 04-07</H1>

In [ ]:
#reserved

In [ ]:
#reserved

In [1]:
from py2neo import Graph,Node,Relationship
import requests as req
import numpy as np
import pandas as pd
import json
import os 

METEO PART (with pressure, wind and temperature that affects flights)

In [53]:
api_key_meteo = '5842eebcfff240981095a321de43a2fa'
# SET LAT AND LONG FROM AN AIRPORT TO CHECK TIMETABLE DELAYS WITH WIND, PRESSURE, TEMP (KELVIN)

In [60]:
test_meteo = req.get('https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(25.92,-80.42,api_key_meteo))
test_meteo.json()

{'coord': {'lon': -80.42, 'lat': 25.92},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 306.56,
  'feels_like': 310.05,
  'temp_min': 305.15,
  'temp_max': 308.15,
  'pressure': 1016,
  'humidity': 55},
 'visibility': 16093,
 'wind': {'speed': 2.6, 'deg': 0},
 'clouds': {'all': 20},
 'dt': 1593795395,
 'sys': {'type': 1,
  'id': 5222,
  'country': 'US',
  'sunrise': 1593772460,
  'sunset': 1593821840},
 'timezone': -14400,
 'id': 4167594,
 'name': 'Palm Springs North',
 'cod': 200}

FLIGHTS PART

In [61]:
test = req.get('https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412').json()[1]
test

{'aircraft': {'iataCode': 'E75L',
  'icao24': 'A01180',
  'icaoCode': 'E75L',
  'regNumber': 'N103SY'},
 'airline': {'iataCode': 'UA', 'icaoCode': 'UAL'},
 'arrival': {'iataCode': 'AUS', 'icaoCode': 'KAUS'},
 'departure': {'iataCode': 'ORD', 'icaoCode': 'KORD'},
 'flight': {'iataNumber': 'UA5352', 'icaoNumber': 'SKW5352', 'number': '5352'},
 'geography': {'altitude': 1424.94,
  'direction': 227.29,
  'latitude': 31.08,
  'longitude': -97.26},
 'speed': {'horizontal': 327.636, 'isGround': 0.0, 'vspeed': -22.248},
 'status': 'en-route',
 'system': {'squawk': None, 'updated': 1593794769}}

In [5]:
flights = req.get('https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412').json()
timetable = req.get('https://aviation-edge.com/v2/public/timetable?key=3abaa5-51c412').json()
routes = req.get('https://aviation-edge.com/v2/public/routes?key=3abaa5-51c412').json()

In [6]:
airplane = req.get('https://aviation-edge.com/v2/public/airplaneDatabase?key=3abaa5-51c412').json() #20k planes
airports = req.get('https://aviation-edge.com/v2/public/airportDatabase?key=3abaa5-51c412').json() #10k
airlines = req.get('https://aviation-edge.com/v2/public/airlineDatabase?key=3abaa5-51c412').json() #13k
countries = req.get('https://aviation-edge.com/v2/public/countryDatabase?key=3abaa5-51c412').json() #252

- Airplanes have an id : regNumber (we need to check if it s unique)
- Airlines and airports have codes (ie AA)
- Geo is available for airports, good for meteo
- 


QUESTIONS PENDING : can we have the location of the aircraft during the flight ? YES but real time


In [24]:
flights[0].keys() #ID PLANE, AIRLINE, START/ARRIVAL AIRPORT, ID FLIGHT ? , GEO, SPEED, STATUS

dict_keys(['aircraft', 'airline', 'arrival', 'departure', 'flight', 'geography', 'speed', 'status', 'system'])

In [7]:
flights[5000]

{'aircraft': {'iataCode': 'BE55',
  'icao24': 'AC5E02',
  'icaoCode': 'BE55',
  'regNumber': 'N8960M'},
 'airline': {'iataCode': '', 'icaoCode': ''},
 'arrival': {'iataCode': 'CRE', 'icaoCode': 'KCRE'},
 'departure': {'iataCode': 'MJX', 'icaoCode': 'KMJX'},
 'flight': {'iataNumber': 'N8960M', 'icaoNumber': 'N8960M', 'number': '960M'},
 'geography': {'altitude': 2407.92,
  'direction': 209.12,
  'latitude': 36.46,
  'longitude': -76.8},
 'speed': {'horizontal': 334.944, 'isGround': 0.0, 'vspeed': 1.188},
 'status': 'en-route',
 'system': {'squawk': None, 'updated': 1594056358}}

In [52]:
len(pd.DataFrame(flights)) #flights atm

10884

REMARK : WE KNOW IF THE PLANE IS GROUND OR NOT, REAL TIME DATA (checked)


In [12]:
timetable[0].keys()

dict_keys(['airline', 'arrival', 'codeshared', 'departure', 'flight', 'status', 'type'])

In [27]:
timetable[0] # AIRLINE, START ARRIVAL DETAILS, FLIGHT ID ? , STATUS

{'airline': {'iataCode': 'TV', 'icaoCode': 'TBA', 'name': 'Tibet Airlines'},
 'arrival': {'actualRunway': '2020-07-03T09:58:00.000',
  'actualTime': '2020-07-03T10:08:00.000',
  'baggage': None,
  'delay': '13',
  'estimatedRunway': '2020-07-03T09:58:00.000',
  'estimatedTime': '2020-07-03T10:08:00.000',
  'gate': None,
  'iataCode': 'NGQ',
  'icaoCode': 'ZUAL',
  'scheduledTime': '2020-07-03T09:55:00.000',
  'terminal': None},
 'codeshared': None,
 'departure': {'actualRunway': '2020-07-03T08:11:00.000',
  'actualTime': '2020-07-03T07:56:00.000',
  'baggage': None,
  'delay': '1',
  'estimatedRunway': '2020-07-03T08:11:00.000',
  'estimatedTime': '2020-07-03T07:56:00.000',
  'gate': None,
  'iataCode': 'LXA',
  'icaoCode': 'ZULS',
  'scheduledTime': '2020-07-03T07:55:00.000',
  'terminal': 'T2'},
 'flight': {'iataNumber': 'TV6008', 'icaoNumber': 'TBA6008', 'number': '6008'},
 'status': 'landed',
 'type': 'arrival'}

In [13]:
routes[0].keys()

dict_keys(['airlineIata', 'airlineIcao', 'arrivalIata', 'arrivalIcao', 'arrivalTerminal', 'arrivalTime', 'codeshares', 'departureIata', 'departureIcao', 'departureTerminal', 'departureTime', 'flightNumber', 'regNumber'])

In [28]:
routes[0] # AIRLINE ID, START/ARRIVAL AIRPORT/TIME, FLIGHT ID, PLANE ID ?

{'airlineIata': '0B',
 'airlineIcao': 'BMS',
 'arrivalIata': 'TRN',
 'arrivalIcao': 'LIMF',
 'arrivalTerminal': None,
 'arrivalTime': '10:45:00',
 'codeshares': None,
 'departureIata': 'OTP',
 'departureIcao': 'LROP',
 'departureTerminal': None,
 'departureTime': '09:15:00',
 'flightNumber': '101',
 'regNumber': ['YR-BAP']}

In [14]:
airplane[0].keys()

dict_keys(['airplaneIataType', 'airplaneId', 'codeIataAirline', 'codeIataPlaneLong', 'codeIataPlaneShort', 'codeIcaoAirline', 'constructionNumber', 'deliveryDate', 'enginesCount', 'enginesType', 'firstFlight', 'hexIcaoAirplane', 'lineNumber', 'modelCode', 'numberRegistration', 'numberTestRgistration', 'planeAge', 'planeClass', 'planeModel', 'planeOwner', 'planeSeries', 'planeStatus', 'productionLine', 'registrationDate', 'rolloutDate'])

In [8]:
airplane # TYPE, ID, REG ID, MODEL, OWNER ? , 

[{'airplaneIataType': 'B737-300',
  'airplaneId': 1,
  'codeIataAirline': '0B',
  'codeIataPlaneLong': 'B733',
  'codeIataPlaneShort': '733',
  'codeIcaoAirline': '',
  'constructionNumber': '23653',
  'deliveryDate': '1986-08-21T22:00:00.000Z',
  'enginesCount': '2',
  'enginesType': 'JET',
  'firstFlight': '1986-08-02T22:00:00.000Z',
  'hexIcaoAirplane': '4A0823',
  'lineNumber': '1260',
  'modelCode': 'B737-377',
  'numberRegistration': 'YR-BAC',
  'numberTestRgistration': '',
  'planeAge': '31',
  'planeClass': None,
  'planeModel': '737',
  'planeOwner': 'Airwork Flight Operations Ltd',
  'planeSeries': '377',
  'planeStatus': 'active',
  'productionLine': 'Boeing 737 Classic',
  'registrationDate': '0000-00-00',
  'rolloutDate': ''},
 {'airplaneIataType': 'B737-400',
  'airplaneId': 2,
  'codeIataAirline': '0B',
  'codeIataPlaneLong': 'B734',
  'codeIataPlaneShort': '734',
  'codeIcaoAirline': '',
  'constructionNumber': '28723',
  'deliveryDate': '1997-05-29T22:00:00.000Z',
  'e

In [15]:
airports[0].keys()

dict_keys(['GMT', 'airportId', 'codeIataAirport', 'codeIataCity', 'codeIcaoAirport', 'codeIso2Country', 'geonameId', 'latitudeAirport', 'longitudeAirport', 'nameAirport', 'nameCountry', 'phone', 'timezone'])

In [11]:
airports[3500] # LOCAL TIME, ID, COUNTRY ID, GEO

{'GMT': '-3',
 'airportId': 3501,
 'codeIataAirport': 'JFR',
 'codeIataCity': 'JFR',
 'codeIcaoAirport': 'BGFH',
 'codeIso2Country': 'GL',
 'geonameId': '6295695',
 'latitudeAirport': 62.00028,
 'longitudeAirport': -49.70028,
 'nameAirport': 'Frederikshab',
 'nameCountry': 'Greenland',
 'phone': '',
 'timezone': 'America/Godthab'}

In [16]:
airlines[0].keys()

dict_keys(['ageFleet', 'airlineId', 'callsign', 'codeHub', 'codeIataAirline', 'codeIcaoAirline', 'codeIso2Country', 'founding', 'iataPrefixAccounting', 'nameAirline', 'nameCountry', 'sizeAirline', 'statusAirline', 'type'])

In [33]:
airlines[0] # ID/CODE, NAME, SIZE ? , STATUS

{'ageFleet': 10.9,
 'airlineId': 1,
 'callsign': 'AMERICAN',
 'codeHub': 'DFW',
 'codeIataAirline': 'AA',
 'codeIcaoAirline': 'AAL',
 'codeIso2Country': 'US',
 'founding': 1934,
 'iataPrefixAccounting': '1',
 'nameAirline': 'American Airlines',
 'nameCountry': 'United States',
 'sizeAirline': 963,
 'statusAirline': 'active',
 'type': 'scheduled'}

In [17]:
countries[0].keys()

dict_keys(['capital', 'codeCurrency', 'codeFips', 'codeIso2Country', 'codeIso3Country', 'continent', 'countryId', 'nameCountry', 'nameCurrency', 'numericIso', 'phonePrefix', 'population'])

In [34]:
countries[0] # ID/CODE COUNTRY, CONTINENT, NAME

{'capital': 'Andorra la Vella',
 'codeCurrency': 'EUR',
 'codeFips': 'AN',
 'codeIso2Country': 'AD',
 'codeIso3Country': 'AND',
 'continent': 'EU',
 'countryId': 1,
 'nameCountry': 'Andorra',
 'nameCurrency': 'Euro',
 'numericIso': '20',
 'phonePrefix': '376',
 'population': '84000'}

Associations : country id - airports

SOME QUERIES WITH NEO4J

In [62]:
'''THEORY :  
WITH {json} AS document
 FOREACH (node in document.data| CREATE(m:`memeber`) SET m = node )'''

'theory :  \nWITH {json} AS document\n FOREACH (node in document.data| CREATE(m:`memeber`) SET m = node )'

In [ ]:
CALL apoc.load.json('https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412')
YIELD value
RETURN value

In [78]:
query = """CALL apoc.load.json('https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412') 
YIELD value 
RETURN value"""
print(query)
json1 = countries
#data = Graph().run('''CREATE (jennifer:Person:US {name: "Jennifer", community: 1, partition: 4})''')
data = Graph().run("WITH {json1} as data YIELD value RETURN value", json1=json1 )

CALL apoc.load.json('https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412') 
YIELD value 
RETURN value


AttributeError: 'NoneType' object has no attribute 'split'

In [80]:
from py2neo import Path, authenticate
# set up authentication parameters
authenticate("localhost:7474", "neo4j", "neo4j")

neo4jUrl = os.environ.get('NEO4J_URL',"http://localhost:7474/db/data/")
graph = Graph(neo4jUrl,secure=False)



ImportError: cannot import name 'authenticate' from 'py2neo' (C:\Users\Megaport\Anaconda3\lib\site-packages\py2neo\__init__.py)

In [82]:
from py2neo.data import Node, Relationship
a = Node("Person", name="Alice")
b = Node("Person", name="Bob")
ab = Relationship(a, "KNOWS", b)
ab


(Alice)-[:KNOWS {}]->(Bob)

_____________________________________________________________________________

OUR REQUESTS

J

In [ ]:
WITH "https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412" AS vol
CALL apoc.load.json(vol) YIELD value 
UNWIND value.airline AS cie 
UNWIND value.aircraft AS plane
MERGE(req:MERGED {q_test:plane.regNumber}) ON CREATE
SET req.statut= value.status, req.compagnie= cie.icaoCode
MATCH (q:MERGED)
WHERE q.statut = 'en-route'
RETURN count(q.statut) AS statuts, q.compagnie
ORDER BY statuts DESC

In [ ]:
WITH "https://aviation-edge.com/v2/public/airportDatabase?key=3abaa5-51c412" AS ap
CALL apoc.load.json(ap) YIELD value 
MERGE(a:AIRPORT {a_test:value.airportId}) ON CREATE
SET a.latitude= value.latitudeAirport, a.longitude= value.longitudeAirport, a.names= value.nameAirport, a.country= value.nameCountry
MERGE (co:Co {cou: a.country})
MERGE (vi:Vi {na: a.names})
CREATE (vi)- [:DANS]->(co)

In [ ]:
#incoming

H

In [ ]:
N